In [ ]:
from source_files import *
from plot_helpers import *
from raster_compare.base import RasterFile
from raster_compare.plots import PlotBase

import math
import pandas as pd

Source files were masked to only ERW watershed boundaries. No mask need to be applied here.

In [ ]:
sfm_snow_free = RasterFile(SFM_SNOW_FREE, band_number=3)
sfm_snow_free_values = sfm_snow_free.band_values()

sfm_snow_on = RasterFile(SFM_SNOW_ON, band_number=3)
sfm_snow_on_values = sfm_snow_on.band_values()

sd_difference_values = sfm_snow_depth_values - aso_snow_depth_values

dem = RasterFile(DEM_SNOW_FREE, band_number=1)
dem_values = dem.band_values()

HILLSHADE_SNOW_ON_AZIMUTH = 100
HILLSHADE_SNOW_FREE_AZIMUTH = 247

hillshade_snow_on = dem.hill_shade(azimuth=100, altitude=47)
hillshade_snow_free =  dem.hill_shade(azimuth=247, altitude=32)

for values in [sfm_snow_free_values, sfm_snow_on_values, sd_difference_values, dem_values]:
    np.ma.masked_where(aso_snow_depth_values.mask, values, copy=False)

assert aso_snow_depth.geo_transform == dem.geo_transform
assert aso_snow_depth.geo_transform == sfm_snow_free.geo_transform == sfm_snow_on.geo_transform

# Analysis

In [ ]:
df = pd.DataFrame({
        'aso_snow_depth': aso_snow_depth_values.ravel(),
        'sfm_snow_depth': sfm_snow_depth_values.ravel(),
        'sd_difference': sd_difference_values.ravel(),
        'elevation': dem_values.ravel(),
        'slope': dem.slope.ravel(),
        'aspect': dem.aspect.ravel(),
        'hillshade_snow_on': hillshade_snow_on.ravel(),
        'hillshade_snow_free': hillshade_snow_free.ravel(),
        'casi_class': casi_classification.ravel(),
    },
    dtype='float32'
)

In [ ]:
df.dropna(inplace=True)
df['elevation'] = df['elevation'].astype('int16')
df['slope'] = df['slope'].astype('int16')
df['aspect'] = df['aspect'].astype('int16')
df['hillshade_snow_on'] = df['hillshade_snow_on'].astype('int16')
df['hillshade_snow_free'] = df['hillshade_snow_free'].astype('int16')
df['casi_class'] = pd.cut(df['casi_class'], CASI_MAPPING, labels=CASI_CLASSES)

positive_sfm = df.query('sfm_snow_depth >= 0')
negative_sfm = df.query('sfm_snow_depth < 0')

## Snow depth by elevation (positive SfM values)

**Snow Depth binning**
* 0.1 m from 0 to 15 m
* 0.5 m from 15 m to 50 m

In [ ]:
bins = np.concatenate((
    np.arange(0, 15.10, 0.10),
    np.arange(15, 50, 0.5),
    [math.ceil(positive_sfm.sfm_snow_depth.max())]
))

elevation_bands = 10
elevation_min = df.elevation.min() - df.elevation.min() % elevation_bands
elevation_max = df.elevation.max() + (elevation_bands - df.elevation.max() % elevation_bands)
elevation_range = np.arange(elevation_min, elevation_max + elevation_bands, elevation_bands)

hist_opts = dict(
    bins=[bins, elevation_range],
    vmin=0,
    vmax=150,
    cmin=1,
)

COLOR_BAR_ATTR = dict(right=0.9, rect=[0.91, 0.125, 0.02, 0.795])
HIST2D_PLOT = dict(ncols=2, sharey=True, sharex=True, figsize=(18,10))

def plot_hist2d(ax, x_data, y_data, title, **kwargs):
    ax.set_facecolor('whitesmoke')
    if len(title) > 0:
        ax.set_xlabel(SNOW_DEPTH_LABEL)
        ax.set_title(title)
    ax.set_xlim(left=-.2)

    return ax.hist2d(x_data, y_data, **kwargs)

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    positive_sfm['sfm_snow_depth'],
    positive_sfm['elevation'],
    'SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)

h2 = plot_hist2d(
    ax2,
    positive_sfm['aso_snow_depth'],
    positive_sfm['elevation'],
    'ASO',
    **hist_opts,
)
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

bin_count_sum = np.count_nonzero(~np.isnan(h1[0].T), axis=1)
ax1.axhline(y=3500, color='orange', linestyle='--')
ax1.scatter(bin_count_sum, elevation_range[:-1])
ax1.set_ylabel(ELEVATION_LABEL)
ax1.set_xlabel('Number of bins')

bin_count_sum = np.count_nonzero(~np.isnan(h2[0].T), axis=1)
ax2.axhline(y=3500, color='orange', linestyle='--')
ax2.scatter(bin_count_sum, elevation_range[:-1])
ax2.set_xlabel('Number of bins')

plt.suptitle('Bin count per elevation band', fontsize=16)
ax1.set_xlim(left=0);

In [ ]:
condition = (positive_sfm.casi_class == 'Vegetation')
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    positive_sfm[condition]['sfm_snow_depth'],
    positive_sfm[condition]['elevation'],
    'SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)

h2 = plot_hist2d(
    ax2,
    positive_sfm[condition]['aso_snow_depth'],
    positive_sfm[condition]['elevation'],
    'ASO',
    **hist_opts,
)

plt.suptitle('Snow Depth in Vegetation')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

In [ ]:
condition = (positive_sfm.casi_class != 'Vegetation')
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    positive_sfm[condition]['sfm_snow_depth'],
    positive_sfm[condition]['elevation'],
    'SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)


h2 = plot_hist2d(
    ax2,
    positive_sfm[condition]['aso_snow_depth'],
    positive_sfm[condition]['elevation'],
    'ASO',
    **hist_opts,
)
plt.suptitle('Snow Depth in Open Areas')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

# Snow Depth Difference

In [ ]:
HIST_BIN_WIDTH = .10
COLORMAP = PlotColor.mpl_colormap

CASI_COLORS = ['dodgerblue', 'forestgreen', 'brown', 'orange']
COLOR_BAR_ATTR = dict(right=0.90, rect=[0.91, 0.22, 0.022, 0.608])

In [ ]:
np.ma.masked_where(
    sfm_snow_depth_values <= 0.0,
    sd_difference_values,
    copy=False
)
plot_data = sd_difference_values[np.isfinite(sd_difference_values)]

classification_plot = np.ma.masked_where(
    np.isfinite(sd_difference_values).mask,
    casi_classification,
).astype(np.int8);

In [ ]:
bins = np.concatenate((
    [math.floor(plot_data.min())],
    np.arange(-2., 2. + HIST_BIN_WIDTH, HIST_BIN_WIDTH),
    [math.ceil(plot_data.max())],
))
bounds = dict(
    norm=colors.BoundaryNorm(
        boundaries=bins, ncolors=COLORMAP.N,
    ),
    cmap=COLORMAP,
)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(24,20))

ax1.set_facecolor('whitesmoke')
ax1.imshow(
    hillshade_snow_on,
    extent=sfm_snow_depth.extent,
    cmap='gray', clim=(1, 255), alpha=0.3,
)
im_data = ax1.imshow(
    sd_difference_values, 
    extent=sfm_snow_depth.extent,
    **bounds
)
ax1.set_title("Snow Depth Differences")
PlotBase.insert_colorbar(ax1, im_data, 'Snow Depth Difference')

ax2.imshow(
    hillshade_snow_on,
    extent=sfm_snow_depth.extent,
    cmap='gray', clim=(1, 255), alpha=0.3,
)
ax2.set_facecolor('whitesmoke')
im_data = ax2.imshow(
    classification_plot, 
    extent=sfm_snow_depth.extent,
    cmap=colors.ListedColormap(CASI_COLORS),
    alpha=0.8,
)
ax2.set_title("Snow Depth Differences - Classification")
PlotBase.insert_colorbar(ax2, im_data, 'Classification');

In [ ]:
high_elevation = np.ma.masked_where(
    dem_values <= 3500,
    sd_difference_values,
)
low_elevation = np.ma.masked_where(
    dem_values > 3500,
    sd_difference_values,
)

fig, (ax1, ax2) = plt.subplots(
    1, 2, 
    sharey=True, 
    figsize=(24,18),
)

ax1.set_facecolor('whitesmoke')
ax1.imshow(
    hillshade_snow_on,
    extent=sfm_snow_depth.extent,
    cmap='gray', clim=(1, 255), alpha=0.3,
)
im_data = ax1.imshow(
    high_elevation, 
    extent=sfm_snow_depth.extent,
    **bounds
)
ax1.set_title("Snow Depth Differences - High Elevation >= 3500m")

ax2.set_facecolor('whitesmoke')
ax2.imshow(
    hillshade_snow_on,
    extent=sfm_snow_depth.extent,
    cmap='gray', clim=(1, 255), alpha=0.3,
)
im_data = ax2.imshow(
    low_elevation, 
    extent=sfm_snow_depth.extent,
    **bounds,
)
ax2.set_title("Snow Depth Differences - Low Elevation < 3500")

PlotBase.insert_colorbar(
    ax2, im_data, 'Snow Depth Difference', **COLOR_BAR_ATTR
);

In [ ]:
high_elevation = np.ma.masked_where(
    np.ma.masked_outside(dem_values, 3100, 3200).mask,
    sd_difference_values,
)
low_elevation = np.ma.masked_where(
    np.ma.masked_outside(dem_values, 3800, 3900).mask,
    sd_difference_values,
)

fig, (ax1, ax2) = plt.subplots(
    1, 2, 
    sharey=True, 
    figsize=(24,18),
)

ax1.set_facecolor('whitesmoke')
ax1.imshow(
    hillshade_snow_free,
    extent=sfm_snow_depth.extent,
    cmap='gray', clim=(1, 255), alpha=0.3,
)
im_data = ax1.imshow(
    high_elevation, 
    extent=sfm_snow_depth.extent,
    **bounds
)
ax1.set_title("Snow Depth Differences - 3100m < Elevation < 3200m")

ax2.set_facecolor('whitesmoke')
ax2.imshow(
    hillshade_snow_free,
    extent=sfm_snow_depth.extent,
    cmap='gray', clim=(1, 255), alpha=0.3,
)
im_data = ax2.imshow(
    low_elevation, 
    extent=sfm_snow_depth.extent,
    **bounds,
)
ax2.set_title("Snow Depth Differences - 3800 < Elevation < 3900")

PlotBase.insert_colorbar(
    ax2, im_data, 'Snow Depth Difference', **COLOR_BAR_ATTR
);

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    1, 2, 
    sharey=True, 
    figsize=(24,18),
)

ax1.set_facecolor('whitesmoke')
ax1.imshow(
    hillshade_snow_free,
    extent=sfm_snow_depth.extent,
    cmap='gray', clim=(1, 255), alpha=0.3,
)
im_data = ax1.imshow(
    sd_difference_values, 
    extent=sfm_snow_depth.extent,
    **bounds
)
ax1.set_title("Snow Depth Differences")

ax2.set_facecolor('whitesmoke')
ax2.imshow(
    hillshade_snow_free,
    extent=sfm_snow_depth.extent,
    cmap='gray', clim=(1, 255), alpha=0.3,
)
im_data = ax2.imshow(
    sfm_snow_free_values - dem_values, 
    extent=sfm_snow_depth.extent,
    **bounds,
)
ax2.set_title("Snow Free - Reference DEM")

PlotBase.insert_colorbar(
    ax2, im_data, 'Snow Depth Difference', **COLOR_BAR_ATTR
);

In [ ]:
data = [
    {
        'data': sfm_snow_free_values - dem_values,
        'label': 'SfM snow free - DEM',
        'color': 'dodgerblue',
    },
]

ax = plot_histogram(data, (-6, 6), figsize=(10, 8))
ax.set_title('Elevation Differences (SFM snow free - DEM)');

# Snow Depth Difference (SfM - ASO)

In [ ]:
cmap = plt.get_cmap("tab20c")
cmap = cmap(np.arange(4)*4, alpha=0.7)

In [ ]:
fig, (ax1) = plt.subplots(
    1, 1, 
    figsize=(8,12),
)

bins = np.arange(math.floor(positive_sfm.sd_difference.min()) -0.1, math.ceil(positive_sfm.sd_difference.max()) + 0.2, 0.2)
stack = []

for casi_class in CASI_CLASSES:
    stack.append(positive_sfm[positive_sfm.casi_class == casi_class].sd_difference)

ax1.hist(
    stack,
    bins=bins,
    label=CASI_CLASSES,
    stacked=True,
    color=cmap,
#     histtype='step',
)
ax1.axvline(0, color='black', linewidth=.4, alpha=1)
ax1.set_xlabel(SNOW_DEPTH_LABEL)
ax1.set_xlim(-5, 5)
ax1.set_ylabel('Count')
ax1.legend(loc='upper left');

In [ ]:
print(f"Mean difference {positive_sfm.sd_difference.mean():.4f}m")
print(f"Median difference {positive_sfm.sd_difference.median():.4f}m")

## Negative by CASI classification

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    1, 2, 
    figsize=(20,8),
    gridspec_kw = {'width_ratios':[2, 3]}
)
fig.subplots_adjust(wspace=.25)
patches, texts, autotexts = ax1.pie(
    negative_sfm.groupby('casi_class').count().aso_snow_depth, 
    labels=CASI_CLASSES, 
    colors=cmap,
    wedgeprops=dict(width=0.5, edgecolor='w'),
    textprops={'fontsize': 14},
    autopct='%1.1f%%'
)
ax1.axis('equal')
ax1.set_title('Percentage by classification')

bins = np.arange(math.floor(negative_sfm.sfm_snow_depth.min()), 0, 0.5)
stack = []

for casi_class in CASI_CLASSES:
    stack.append(negative_sfm[negative_sfm.casi_class == casi_class].sfm_snow_depth)

ax2.hist(
    stack,
    bins=bins,
    label=CASI_CLASSES,
    stacked=True,
    color=cmap,
#     histtype='step',
)
ax2.set_xlabel(SNOW_DEPTH_LABEL)
ax2.set_xlim(-30, bins.max())
ax2.set_ylabel('Count')
ax2.legend(loc='upper left');

## SfM negative snow depth values with aspect

In [ ]:
bins = np.concatenate((
    np.arange(0, -30.1, -0.2),
    np.arange(-30, -40, -0.5),
    [math.floor(negative_sfm.sfm_snow_depth.min())]
))
bins = np.flip(bins)

aspect_range = np.arange(0, 361, 1)

hist_opts = dict(
    bins=[aspect_range, bins],
    vmin=0,
    vmax=100,
    cmin=1,
)

In [ ]:
fig = plt.figure(figsize=(15,8))
ax = fig.gca()
ax.set_facecolor('whitesmoke')
ha = ax.hist2d(
    negative_sfm['aspect'],
    negative_sfm['sfm_snow_depth'],
    **hist_opts,
)
PlotBase.insert_colorbar(ax, ha[3], 'count')
ax.set_ylabel(SNOW_DEPTH_LABEL)
ax.set_xlabel('Aspect')
ax.set_title('Negative values with aspect');

In [ ]:
aspect_stats = negative_sfm[['sfm_snow_depth', 'aspect']].groupby('aspect')

aspect_count = aspect_stats.count()
aspect_medians = aspect_stats.median()
aspect_means = aspect_stats.mean()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(14, 24))
ax1.scatter(aspect_count.index, aspect_count.sfm_snow_depth)
ax1.set_title('Aspect count')

ax2.scatter(aspect_medians.index, aspect_medians.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference by aspect');

ax3.scatter(aspect_means.index, aspect_means.sfm_snow_depth)
ax3.set_xlim(aspect_range.min(), aspect_range.max())
ax3.set_ylabel('Mean SD difference')
ax3.set_xlabel('Aspect in degree')
ax3.set_title('Mean difference by aspect');

In [ ]:
aspect_count.sort_values('sfm_snow_depth', ascending=False).head(10).sort_index()

**NOTE**: Vegetation and Water category masked out.

In [ ]:
vegetation_free = negative_sfm.query('casi_class != "Vegetation" and casi_class != "Water"')
vegetation_free_aspects = vegetation_free.groupby('aspect')
vegetation_free_aspects_median = vegetation_free_aspects.median()
vegetation_free_aspects_count = vegetation_free_aspects.count()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15,24))
ax1.set_facecolor('whitesmoke')
ha = ax1.hist2d(
    vegetation_free['aspect'],
    vegetation_free['sfm_snow_depth'],
    **hist_opts,
)
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with aspect - No Vegetation')

ax2.scatter(vegetation_free_aspects_median.index, vegetation_free_aspects_median.sfm_snow_depth)
ax2.set_title('Mean difference by aspect - No Vegetation')
ax2.axvline(HILLSHADE_SNOW_ON_AZIMUTH, color='goldenrod', linestyle='--')
ax2.annotate('Snow On\n Azimuth', xy=(HILLSHADE_SNOW_ON_AZIMUTH + 1, -8))
ax2.axvline(HILLSHADE_SNOW_ON_AZIMUTH + 180, color='dimgrey', linestyle='--')
ax2.axvline(HILLSHADE_SNOW_FREE_AZIMUTH, color='goldenrod', linestyle='--')
ax2.annotate('Snow Free\n Azimuth', xy=(HILLSHADE_SNOW_FREE_AZIMUTH + 1, -8))
ax2.axvline(HILLSHADE_SNOW_FREE_AZIMUTH % 180, color='dimgrey', linestyle='--')
ax2.set_ylabel('Median SD difference')

ax3.scatter(vegetation_free_aspects_count.index, vegetation_free_aspects_count.sfm_snow_depth)
ax3.set_title('Count of aspects - No Vegetation');
ax3.set_ylabel('Median SD difference')

ax3.set_xlim(aspect_range.min(), aspect_range.max())
ax3.set_xlabel('Aspect in degree');

# PlotBase.insert_colorbar(ax1, ha[3], 'count');

## SfM negative snow depth values with slope

In [ ]:
slope_range = np.arange(0, 91, 1)

hist_opts = dict(
    bins=[slope_range, bins],
    vmin=0,
    vmax=150,
    cmin=1,
)

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.gca()
ax.set_facecolor('whitesmoke')
hs = ax.hist2d(
    negative_sfm['slope'],
    negative_sfm['sfm_snow_depth'],
    **hist_opts
)
PlotBase.insert_colorbar(ax, hs[3], 'count')
ax.set_ylabel(SNOW_DEPTH_LABEL)
ax.set_xlabel('Slope')
ax.set_title('Negative values with slope');

In [ ]:
slope_stats = negative_sfm[['sfm_snow_depth', 'slope']].groupby('slope')
slope_count = slope_stats.count()
slope_median = slope_stats.median()
slope_means = slope_stats.mean()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15, 24))

ax1.scatter(slope_count.index, slope_count.sfm_snow_depth)
ax1.set_title('Slope Count')

ax2.scatter(slope_median.index, slope_median.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference by slope')

ax3.scatter(slope_means.index, slope_means.sfm_snow_depth)
ax3.set_xlim(slope_range.min(), slope_range.max())
ax3.set_xlabel('Slope in degree')
ax3.set_ylabel('Mean SD difference')
ax3.set_title('Mean difference by slope');

In [ ]:
slope_count.query('sfm_snow_depth > 20000').sort_index()

In [ ]:
vegetation_free_slopes = vegetation_free.groupby('slope')
vegetation_free_slopes_median = vegetation_free_slopes.median()
vegetation_free_slopes_mean = vegetation_free_slopes.mean()
vegetation_free_slopes_count = vegetation_free_slopes.count()

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=True, figsize=(15,32))

ax1.set_facecolor('whitesmoke')
hs = ax1.hist2d(
    vegetation_free['slope'],
    vegetation_free['sfm_snow_depth'],
    **hist_opts
)
PlotBase.insert_colorbar(ax1, hs[3], 'count')
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with slope - No Vegetation')

ax2.scatter(vegetation_free_slopes_median.index, vegetation_free_slopes_median.sfm_snow_depth)
ax2.axhline(-1, color='orange', linestyle='--', alpha=0.8)
ax2.axhline(-2, color='orange', linestyle='--', alpha=0.8)
ax2.axvline(51, color='orange', linestyle='--', alpha=0.8)
ax2.axvline(75, color='orange', linestyle='--', alpha=0.8)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference by slope - No vegetation')

ax3.scatter(vegetation_free_slopes_mean.index, vegetation_free_slopes_mean.sfm_snow_depth)
ax3.axvline(51, color='orange', linestyle='--', alpha=0.8)
ax3.axvline(75, color='orange', linestyle='--', alpha=0.8)
ax3.set_ylabel('Mean SD difference')
ax3.set_title('Mean difference by slope - No vegetation')

ax4.scatter(vegetation_free_slopes_count.index, vegetation_free_slopes_count.sfm_snow_depth)
ax4.axvline(51, color='orange', linestyle='--', alpha=0.8)
ax4.set_title('Count of slope angels - No vegetation')

ax4.set_xlabel('Slope in degree')
ax4.set_xlim(slope_range.min(), slope_range.max());

## Hillshade comparison (No Vegetation)

## Snow free SfM

Azimuth: 247

Altitude: 32

In [ ]:
HILLSHADE_MIN = 0
HILLSHADE_MAX = 255

hillshade_range = np.arange(HILLSHADE_MIN, HILLSHADE_MAX + 1, 1)

hist_opts = dict(
    bins=[hillshade_range, bins],
    vmin=0,
    vmax=150,
    cmin=1,
)

In [ ]:
hillshade_snow_free = vegetation_free.groupby('hillshade_snow_free')
hillshade_snow_free_median = hillshade_snow_free.median()
hillshade_snow_free_count = hillshade_snow_free.count()

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15,24))

ax1.set_facecolor('whitesmoke')
ax1.set_facecolor('whitesmoke')
ha = ax1.hist2d(
    negative_sfm.hillshade_snow_free,
    negative_sfm.sfm_snow_depth,
    **hist_opts,
)
PlotBase.insert_colorbar(ax1, ha[3], 'count')
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with Snow Free hillshade')

ax2.scatter(hillshade_snow_free_median.index, hillshade_snow_free_median.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference')

ax3.scatter(hillshade_snow_free_count.index, hillshade_snow_free_count.sfm_snow_depth)
ax3.set_ylabel('Count')
ax3.set_title('Count by hillshade')

ax3.set_xlim(HILLSHADE_MIN, HILLSHADE_MAX)
ax3.set_xlabel('Hillshade');

## SfM snow on

Azimuth: 100

Altitude: 47

In [ ]:
hillshade_snow_on = vegetation_free.groupby('hillshade_snow_on')
hillshade_snow_on_median = hillshade_snow_on.median()
hillshade_snow_on_count = hillshade_snow_on.count()

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15,24))

ax1.set_facecolor('whitesmoke')
ax1.set_facecolor('whitesmoke')
ha = ax1.hist2d(
    negative_sfm.hillshade_snow_on,
    negative_sfm.sfm_snow_depth,
    **hist_opts,
)
PlotBase.insert_colorbar(ax1, ha[3], 'count')
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with Snow On hillshade')

ax2.scatter(hillshade_snow_on_median.index, hillshade_snow_on_median.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference')

ax3.scatter(hillshade_snow_on_count.index, hillshade_snow_on_count.sfm_snow_depth)
ax3.set_ylabel('Count')
ax3.set_title('Count by hillshade')

ax3.set_xlim(HILLSHADE_MIN, HILLSHADE_MAX)
ax3.set_xlabel('Hillshade');